Modo duende con segmentación de fondo

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import random
import math
import time
from mediapipe.tasks.python import vision
from mediapipe.tasks.python import BaseOptions

# Inicializamos MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Lista de la segmentacion
segmentation_result_list = []

# Procesamos los resultados de la segmentación
def segmentation_callback(result, output_image, timestamp_ms):
    segmentation_result_list.append(result)

# Configuramos las opciones del segmentador
options = vision.ImageSegmenterOptions(
    base_options=BaseOptions(model_asset_path="models/selfie_segmenter.tflite"), 
    output_category_mask=True,
    running_mode=vision.RunningMode.LIVE_STREAM, 
    result_callback=segmentation_callback
)

# Creamos el segmentador
segmenter = vision.ImageSegmenter.create_from_options(options)

cap = cv2.VideoCapture(0)

# Cargar las imágenes de las orejas y los emojis de dinero
left_ear_img = cv2.imread('images/orejaI.png', cv2.IMREAD_UNCHANGED) 
right_ear_img = cv2.imread('images/orejaD.png', cv2.IMREAD_UNCHANGED)
money_emoji = cv2.imread('images/dinero.png', cv2.IMREAD_UNCHANGED)

def euclidean_distance(p1, p2):
    return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

# Lista para almacenar los emojis de dinero
falling_emoji = []

# Representamos el dinero
class FallingEmoji:
    def __init__(self, x, y, speed, time_to_live):
        self.x = x
        self.y = y
        self.speed = speed
        self.time_to_live = time_to_live  # Tiempo de vida del emoji
        self.creation_time = time.time()  # Momento en que se creó el emoji

    def update(self):
        self.y += self.speed  # Movimiento hacia abajo
        if self.y > 480:  # Si el emoji se sale de la pantalla, lo reubicamos
            self.y = 0
            self.x = random.randint(0, 640)

        if time.time() - self.creation_time > self.time_to_live:
            return False  # El emoji debe eliminarse
        return True  # El emoji sigue existiendo


            
# Procesamos los resultados y agregar las orejas y el emoji de dinero
def process_frame(frame, face_mesh):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    points_data = {}

    if results.multi_face_landmarks is not None:
        for face_landmarks in results.multi_face_landmarks:
            points = [234, 454, 0, 17]  # Añadir los puntos para las orejas (234 y 454) y los puntos de la boca (0 y 17)

            for point in points:
                if point < len(face_landmarks.landmark):
                    x = face_landmarks.landmark[point].x
                    y = face_landmarks.landmark[point].y
                    z = face_landmarks.landmark[point].z

                    x_pixel = int(x * frame.shape[1])
                    y_pixel = int(y * frame.shape[0])

                    points_data[point] = {'x': x_pixel, 'y': y_pixel, 'z': z}

            # Colocar las orejas (puntos 234 y 454 para la oreja izquierda y derecha)
            if 234 in points_data:
                left_ear_position = points_data[234]
                left_ear_resized = cv2.resize(left_ear_img, (100, 100))  # Aumenta el tamaño de la oreja
                lx, ly = left_ear_position['x'], left_ear_position['y']

                lx -= 80 
                ly -= 60

                # Superponemos la oreja izquierda
                for c in range(0, 3):  # Para las 3 capas (RGB)
                    frame[ly:ly + left_ear_resized.shape[0], lx:lx + left_ear_resized.shape[1], c] = \
                        left_ear_resized[:, :, c] * (left_ear_resized[:, :, 3] / 255.0) + \
                        frame[ly:ly + left_ear_resized.shape[0], lx:lx + left_ear_resized.shape[1], c] * (1.0 - left_ear_resized[:, :, 3] / 255.0)

            if 454 in points_data:
                right_ear_position = points_data[454]
                right_ear_resized = cv2.resize(right_ear_img, (100, 100))  # Aumentamos el tamaño de la oreja
                rx, ry = right_ear_position['x'], right_ear_position['y']

                rx -= 15  
                ry -= 60

                # Superponemos la oreja derecha
                for c in range(0, 3):  # Para las 3 capas (RGB)
                    frame[ry:ry + right_ear_resized.shape[0], rx:rx + right_ear_resized.shape[1], c] = \
                        right_ear_resized[:, :, c] * (right_ear_resized[:, :, 3] / 255.0) + \
                        frame[ry:ry + right_ear_resized.shape[0], rx:rx + right_ear_resized.shape[1], c] * (1.0 - right_ear_resized[:, :, 3] / 255.0)

            # Comprobamos si la boca está abierta (puntos 0 y 17)
            if 0 in points_data and 17 in points_data:
                mouth_distance = euclidean_distance(
                    (points_data[0]['x'], points_data[0]['y']),
                    (points_data[17]['x'], points_data[17]['y'])
                )
                threshold = 40 
                if mouth_distance > threshold:
                    if random.random() < 0.3:
                        new_emoji = FallingEmoji(random.randint(0, frame.shape[1] - 80), 0, random.randint(2, 5), time_to_live=5)
                        falling_emoji.append(new_emoji)
                        
            # Actualizar la posición de los emojis que caen
            falling_emoji[:] = [emoji for emoji in falling_emoji if emoji.update()]
            
            # Actualizar la posición de los emojis que caen
            for emoji in falling_emoji:
                emoji.update()
                # Redimensionamos la imagen del emoji
                emoji_resized = cv2.resize(money_emoji, (80, 50))

                # Superponemos el emoji sobre el frame
                for c in range(0, 3):
                    if emoji.y + emoji_resized.shape[0] <= frame.shape[0] and emoji.x + emoji_resized.shape[1] <= frame.shape[1]:
                        frame[emoji.y:emoji.y + emoji_resized.shape[0], emoji.x:emoji.x + emoji_resized.shape[1], c] = \
                            emoji_resized[:, :, c] * (emoji_resized[:, :, 3] / 255.0) + \
                            frame[emoji.y:emoji.y + emoji_resized.shape[0], emoji.x:emoji.x + emoji_resized.shape[1], c] * (1.0 - emoji_resized[:, :, 3] / 255.0)

    return frame

# Creamos el FaceMesh y procesamos los frames
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertimos el frame a RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

        # Obtenemos los resultados del segmentador
        segmenter.segment_async(frame_rgb, time.time_ns() // 1_000_000)

        if len(segmentation_result_list) > 0:
            segmentation_result = segmentation_result_list[0]
            category_mask = segmentation_result.category_mask
            category_mask_np = category_mask.numpy_view()

            # Creamos la máscara binaria
            mask = (category_mask_np > 0).astype(np.uint8) * 255
            mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]))

            # Reemplazamos el fondo con la máscara
            background_image = cv2.imread("images/fondo.jpg")
            background_resized = cv2.resize(background_image, (frame.shape[1], frame.shape[0]))
            masked_background = cv2.bitwise_and(background_resized, background_resized, mask=mask_resized)
            masked_frame = cv2.bitwise_and(frame, frame, mask=255-mask_resized)
            final_frame = cv2.add(masked_background, masked_frame)

            # Procesamos el frame para agregar las orejas y el emoji de dinero
            final_frame = process_frame(final_frame, face_mesh)

            # Mostramos el resultado final
            cv2.imshow("final_frame", final_frame)

        # Limpiamos la lista de resultados de segmentación
        segmentation_result_list.clear()

        if cv2.waitKey(1) & 0xFF == 27:  # 'Esc' para salir
            break

# Liberamos la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mediapipe\tasks\python\vision\image_segmenter.py:158: UserWarning: MessageFactory class is deprecated. Please use GetMessageClass() instead of MessageFactory.GetPrototype. MessageFactory class will be removed after 2024.
  graph_config = self._runner.get_graph_config()
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
